In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pycolmap
import sqlite3
import visu3d as v3d

import burybarrel.colmap_util as cutil

In [6]:
image_dir = Path("../data/dive-data/Dive8/clips/barrelddt1")

reconstr_path = Path(f"../results/{image_dir.name}-reconstr")
output_path = reconstr_path / "colmap-out"
output_path.mkdir(parents=True, exist_ok=True)
(reconstr_path / "openmvs-out").mkdir(exist_ok=True)
(reconstr_path / "masks").mkdir(exist_ok=True)
mvs_path = output_path / "mvs"
database_path = output_path / "database.db"

In [7]:
# camera = pycolmap.Camera(
#     model="RADIAL",
#     width=1920,
#     height=875,
#     params=[1246, 960, 420, -0.123, -0.015]
# )
# camera = pycolmap.Camera(
#     model="SIMPLE_PINHOLE",
#     width=1920,
#     height=875,
#     # params=[1246, 960, 420],
#     params=[500, 960, 420],
# )
camera = pycolmap.Camera(
    model="PINHOLE",
    width=1920,
    height=875,
    params=[1246, 1246, 960, 420],
    # params=[500, 500, 960, 420],
)

In [ ]:
pycolmap.extract_features(
    database_path,
    image_dir,
    camera_model=camera.model.name,
    reader_options={"camera_model": camera.model.name, "camera_params": camera.params_to_string()},
    sift_options={
        "domain_size_pooling": True,
        "edge_threshold":  5.0,
        "peak_threshold":  1 / 200,
        "max_num_orientations": 3,
        "num_octaves": 8,
        "octave_resolution": 6,
        "num_threads": 4,
        "estimate_affine_shape": True,
        "dsp_max_scale": 6.0,
        "dsp_min_scale": 0.08,
        "dsp_num_scales": 20,
    }
)

In [ ]:
images = cutil.get_images(database_path)
all_keypoints, all_descriptors = cutil.get_features(database_path)

idx = 0
img = images[idx]
kp = all_keypoints[idx]
plt.imshow(plt.imread(image_dir / img.name))
plt.scatter(kp[:, 0], kp[:, 1], s=1, c="r")
plt.show()

In [ ]:
images

In [ ]:
pycolmap.match_exhaustive(
    database_path,
    matching_options={"block_size": 10},
    verification_options={
        "detect_watermark": False
    }
)

In [ ]:
maps = pycolmap.incremental_mapping(
    database_path, image_dir, output_path,
    options={
        "ba_global_function_tolerance": 1e-2,
        "ba_local_function_tolerance": 1e-2,
        "init_num_trials": 400,
        # "init_image_id1": 1,
        # "init_image_id2": 2,
    }
)

In [ ]:
maps

In [ ]:
# dense reconstruction
pycolmap.undistort_images(
    mvs_path,
    output_path / "0",
    image_dir,
    output_type="COLMAP",
)

In [ ]:
reconstruction = pycolmap.Reconstruction(output_path / "0")
print(reconstruction.summary())

In [ ]:
reconstruction.images

In [ ]:
pts, cols = cutil.get_pc(reconstruction)
ptcloud = v3d.Point3d(p=pts, rgb=cols)
ptcloud.fig